## Setup: Import Libraries and Initialize Spark

**⚠️ Important**: If you get a `FileNotFoundError` about `./bin/spark-submit`, you need to:
1. Ensure `JAVA_HOME` is set in your `.env` file (see README)
2. **Restart the kernel** (Kernel → Restart Kernel)
3. Run this cell again to load the environment variables

In [1]:
import sys
import os

# CRITICAL: Set JAVA_HOME before any PySpark imports
# This must be the very first thing we do
java_home = '/Library/Java/JavaVirtualMachines/amazon-corretto-17.jdk/Contents/Home'
os.environ['JAVA_HOME'] = java_home
print(f"✅ JAVA_HOME set to: {java_home}")

# Also set SPARK_HOME if needed (PySpark will find it automatically, but let's be explicit)
import site
site_packages = site.getsitepackages()[0]
spark_home = os.path.join(site_packages, 'pyspark')
if os.path.exists(spark_home):
    os.environ['SPARK_HOME'] = spark_home
    print(f"✅ SPARK_HOME set to: {spark_home}")

from dotenv import load_dotenv

# Load other environment variables from .env file
project_root = os.path.dirname(os.getcwd())
env_path = os.path.join(project_root, '.env')
if os.path.exists(env_path):
    load_dotenv(env_path, override=False)  # Don't override JAVA_HOME we just set
    print(f"✅ Loaded environment from: {env_path}")

# Add project root to path
sys.path.insert(0, project_root)

# NOW import PySpark - JAVA_HOME is already set
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, upper, lower, concat, when, year, month
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

# Import project utilities
from utils.spark_session import get_spark_session, stop_spark_session, get_data_dir
from utils.data_generator import generate_all_datasets

print("✅ Libraries imported successfully!")
print(f"   Python: {sys.version.split()[0]}")
print(f"   Project root: {project_root}")


✅ JAVA_HOME set to: /Library/Java/JavaVirtualMachines/amazon-corretto-17.jdk/Contents/Home
✅ SPARK_HOME set to: /Users/r39132/Projects/learn-pyspark-1/.venv/lib/python3.12/site-packages/pyspark
✅ Loaded environment from: /Users/r39132/Projects/learn-pyspark-1/.env
✅ Libraries imported successfully!
   Python: 3.12.0
   Project root: /Users/r39132/Projects/learn-pyspark-1


In [2]:
# Verify Java is accessible
import subprocess
try:
    java_version = subprocess.run(
        [os.path.join(os.environ['JAVA_HOME'], 'bin', 'java'), '-version'],
        capture_output=True,
        text=True,
        timeout=5
    )
    print("✅ Java is accessible:")
    print(java_version.stderr.split('\n')[0])
except Exception as e:
    print(f"❌ Error accessing Java: {e}")
    print(f"   JAVA_HOME: {os.environ.get('JAVA_HOME', 'NOT SET')}")
    print(f"   Try: export JAVA_HOME=$(jenv prefix 17.0.8.1)")

✅ Java is accessible:
openjdk version "17.0.8.1" 2023-08-22 LTS


In [ ]:
# Initialize Spark Session
spark = get_spark_session("Job 1: DataFrame Basics")
data_dir = get_data_dir()

print(f"✅ Spark session created!")
print(f"📊 Spark UI: http://localhost:4040")
print(f"📁 Data directory: {data_dir}")

In [ ]:
# Generate sample data if it doesn't exist
users_csv = os.path.join(data_dir, "users.csv")
if not os.path.exists(users_csv):
    print("📁 Sample data not found. Generating...")
    generate_all_datasets(data_dir)
    print("✅ Sample data generated!")
else:
    print("✅ Sample data already exists.")

## LESSON 1: Creating DataFrames from Lists

This is useful for quick testing and learning, but in production you'll typically read from files or databases.

In [ ]:
# Simple list of tuples
data = [
    (1, "Alice", 28, "New York"),
    (2, "Bob", 35, "San Francisco"),
    (3, "Charlie", 42, "Seattle"),
    (4, "Diana", 31, "Boston")
]

# Create DataFrame with column names
df = spark.createDataFrame(data, ["id", "name", "age", "city"])

print("📊 Simple DataFrame:")
df.show()

# Print schema - shows column names and types
print("📋 Schema (inferred automatically):")
df.printSchema()

## LESSON 2: Creating DataFrames with Explicit Schema

Defining schema explicitly is better for:
- Performance (no inference needed)
- Type safety (you control the types)
- Clarity (self-documenting code)

In [ ]:
# Define schema explicitly
schema = StructType([
    StructField("id", IntegerType(), False),        # False = not nullable
    StructField("name", StringType(), False),
    StructField("age", IntegerType(), True),        # True = nullable
    StructField("salary", DoubleType(), True)
])

data = [
    (1, "Alice", 28, 75000.0),
    (2, "Bob", 35, 85000.0),
    (3, "Charlie", None, 92000.0),  # Note: age is nullable
    (4, "Diana", 31, None)           # salary is nullable
]

df_schema = spark.createDataFrame(data, schema)

print("📊 DataFrame with explicit schema:")
df_schema.show()

print("📋 Schema (explicitly defined):")
df_schema.printSchema()

## LESSON 3: Reading Data from CSV Files

CSV is common but not the most efficient format. Always specify options like header, inferSchema, delimiter.

In [ ]:
users_path = os.path.join(data_dir, "users.csv")

# Read CSV with options
users_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(users_path)

print(f"📁 Read from: {users_path}")
print(f"📊 Row count: {users_df.count()}")
print("\n🔍 First 10 rows:")
users_df.show(10)

print("📋 Inferred schema:")
users_df.printSchema()

## LESSON 4: Basic DataFrame Transformations

Key transformations:
- `select()`: Choose specific columns
- `filter()`/`where()`: Filter rows based on conditions
- `withColumn()`: Add or modify columns
- `drop()`: Remove columns

In [ ]:
# SELECT: Choose specific columns
print("🔹 select() - Choose columns:")
users_df.select("name", "email", "age").show(5)

In [ ]:
# SELECT with column expressions
print("🔹 select() with expressions:")
users_df.select(
    col("name"),
    col("age"),
    col("city"),
    col("country")
).show(5)

In [ ]:
# FILTER: Keep only rows that match condition
print("🔹 filter() - Users age > 30:")
users_df.filter(col("age") > 30).show(5)

In [ ]:
# Multiple filter conditions (AND)
print("🔹 filter() with multiple conditions (AND):")
users_df.filter(
    (col("age") > 25) & (col("country") == "USA")
).show(5)

In [ ]:
# Multiple filter conditions (OR)
print("🔹 filter() with OR condition:")
users_df.filter(
    (col("country") == "USA") | (col("country") == "UK")
).show(5)

In [ ]:
# WHERE: Alternative to filter (same functionality)
print("🔹 where() - Same as filter:")
users_df.where("age > 40").show(5)  # Can use SQL-like strings

In [ ]:
# WITH_COLUMN: Add new column or modify existing
print("🔹 withColumn() - Add age_group column:")
df_with_group = users_df.withColumn(
    "age_group",
    when(col("age") < 30, "Young")
    .when(col("age") < 50, "Middle")
    .otherwise("Senior")
)
df_with_group.select("name", "age", "age_group").show(10)

In [ ]:
# Multiple withColumn operations (chaining)
print("🔹 Chain multiple withColumn():")
df_enhanced = users_df \
    .withColumn("full_name", upper(col("name"))) \
    .withColumn("email_lower", lower(col("email"))) \
    .withColumn("age_plus_10", col("age") + 10)

df_enhanced.select("name", "full_name", "age", "age_plus_10").show(5)

In [ ]:
# DROP: Remove columns
print("🔹 drop() - Remove columns:")
df_reduced = users_df.drop("user_id", "signup_date")
print(f"Original columns: {len(users_df.columns)}, After drop: {len(df_reduced.columns)}")
df_reduced.show(5)

## LESSON 5: Column Operations and Expressions

Column operations are the heart of DataFrame transformations. Learn different ways to reference and manipulate columns.

In [ ]:
# Column arithmetic
print("🔹 Column arithmetic:")
users_df.select(
    col("name"),
    col("age"),
    (col("age") + 10).alias("age_in_10_years"),
    (col("age") * 12).alias("age_in_months")
).show(5)

In [ ]:
# String operations
print("🔹 String operations:")
users_df.select(
    col("name"),
    upper(col("name")).alias("name_upper"),
    lower(col("name")).alias("name_lower"),
    concat(col("name"), lit(" - "), col("city")).alias("name_city")
).show(5)

In [ ]:
# Conditional expressions with when/otherwise
print("🔹 Conditional expressions (when/otherwise):")
users_df.select(
    col("name"),
    col("country"),
    when(col("country") == "USA", "Domestic")
    .when(col("country") == "Canada", "Domestic")
    .otherwise("International")
    .alias("market")
).show(10)

In [ ]:
# Working with dates (if available)
if "signup_date" in users_df.columns:
    print("🔹 Date operations:")
    users_df.select(
        col("name"),
        col("signup_date"),
        year(col("signup_date")).alias("signup_year"),
        month(col("signup_date")).alias("signup_month")
    ).show(5)

## LESSON 6: Reading and Writing Different Formats

Formats:
- **CSV**: Human-readable, but slow and not type-safe
- **JSON**: Good for nested data, but verbose
- **Parquet**: Columnar format, fast, compressed (RECOMMENDED)

In [ ]:
# Read JSON
print("🔹 Reading JSON:")
products_df = spark.read.option("multiLine", "true").json(os.path.join(data_dir, "products.json"))
print(f"Products JSON: {products_df.count()} rows")
products_df.show(5)

In [ ]:
# Write as Parquet (best practice for Spark)
print("🔹 Writing as Parquet:")
output_path = os.path.join(data_dir, "users_parquet")
users_df.write \
    .mode("overwrite") \
    .parquet(output_path)
print(f"✓ Written to: {output_path}")

In [ ]:
# Read back from Parquet
print("🔹 Reading Parquet:")
users_parquet = spark.read.parquet(output_path)
print(f"Read back: {users_parquet.count()} rows")
users_parquet.show(5)

# Compare file sizes (Parquet is much smaller)
csv_size = os.path.getsize(os.path.join(data_dir, "users.csv"))
print(f"\n📦 CSV size: {csv_size:,} bytes")
print("   Parquet is typically 5-10x smaller and much faster to read!")

In [ ]:
# Writing with partitioning (important for large datasets)
print("🔹 Writing with partitioning:")
partitioned_path = os.path.join(data_dir, "users_partitioned")
users_df.write \
    .mode("overwrite") \
    .partitionBy("country") \
    .parquet(partitioned_path)
print(f"✓ Written partitioned data to: {partitioned_path}")
print("   Partitioning helps with query performance on large datasets!")

## LESSON 7: DataFrame Actions (Trigger Computation)

Actions trigger computation. Transformations are lazy, actions are eager.

Common actions:
- `show()`: Display data
- `count()`: Count rows
- `collect()`: Bring all data to driver (BE CAREFUL!)
- `take()`: Take first N rows
- `first()`: Get first row

In [ ]:
print("🔹 show() - Display data:")
users_df.show(5)

In [ ]:
print("🔹 count() - Count rows:")
row_count = users_df.count()
print(f"Total rows: {row_count}")

In [ ]:
print("🔹 first() - Get first row:")
first_row = users_df.first()
print(f"First row: {first_row}")

In [ ]:
print("🔹 take() - Get first N rows:")
first_3 = users_df.take(3)
print(f"First 3 rows: {len(first_3)} rows")
for row in first_3:
    print(f"  {row}")

In [ ]:
print("🔹 describe() - Statistical summary:")
users_df.describe("age").show()

## Summary

### ✅ What You Learned:
- ✓ Creating DataFrames from lists and files
- ✓ Defining explicit schemas
- ✓ Basic transformations: select, filter, withColumn
- ✓ Column operations and expressions
- ✓ Reading and writing different formats (CSV, JSON, Parquet)
- ✓ Understanding actions vs transformations

### 🎯 Next Step:
Run Notebook 2 to learn about aggregations!

## Cleanup (Optional)

Uncomment and run this cell to stop the Spark session when you're done:

In [ ]:
# stop_spark_session(spark)
# print("✅ Spark session stopped.")